In [1]:
from main import*

C:\Users\Administrator\Documents\yh.wang\2\diwang_feature_exrtaction.py:194: RuntimeWarning: invalid value encountered in true_divide
  W = Weight / np.tile(reshape(sum(Weight, 2), (Num, TestData.shape[1] - 1, 1)), (1, 1, Nu))


In [17]:
class CFG:
    seed=42
    input_dim=3
    input_feature=18
    num_workers=2
    hidden_size=64
    batch_size=2048
    num_classes=1
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    optimizer="Adam"
    scheduler="ExponentialLR"
    lr=1e-1
    epoches=10000
    kernel=True
    print_training_process=True
    sc_Gamma=0.998#指数型学习率衰减曲线
    decay = 1e-5
    num_in_feature_classes=3

In [18]:
def fusion_tG(tG, dev_tG, dev2_tG):
    input_features = []
    for unit in range(len(tG)):
        feature_unit = np.vstack([tG[unit],
                                  dev_tG[unit],
                                  dev2_tG[unit]]).reshape(-1)
        input_features.append(feature_unit)
    return np.array(input_features)
tGG=fusion_tG(tG_delete,dev_tG_delete,dev2_tG_delete)
classifier_in=KMeans(n_clusters=CFG.num_in_feature_classes,random_state=CFG.seed).fit(tGG)

In [19]:
class TestingFeature(Dataset):
    def __init__(self, tG, dev_tG, dev2_tG, RUL, classifier):  # ,labels):
        self.tG = tG
        self.dev_tG = dev_tG
        self.dev2_tG = dev2_tG
        self.RUL = RUL
        self.classifier=classifier
        self.input=fusion_tG(tG, dev_tG, dev2_tG)
        self.labels=self.classifier.predict(self.input)
        # self.labels=labels


    def __len__(self):
        return len(self.input)

    def __getitem__(self, idx):
        data = {
            "input": torch.tensor(self.input[idx], dtype=torch.float),
            "RUL": torch.tensor(self.RUL[idx], dtype=torch.int64),
            "label": torch.tensor(self.labels[idx], dtype=torch.int64)
        }

        return data

In [20]:
test_dataset = TestingFeature(tG_delete[0:2],
                              dev_tG_delete[0:2],
                              dev2_tG_delete[0:2],
                              RUL_delete[0:2],
                              classifier_in)
test_dataset[0]

{'input': tensor([0.1196, 0.1020, 0.0999, 0.0757, 0.1094, 0.0801, 0.0546, 0.0567, 0.0629,
         0.0891, 0.0954, 0.1050, 0.0326, 0.0295, 0.0327, 0.0489, 0.0550, 0.0529]),
 'RUL': tensor(44),
 'label': tensor(1)}

In [23]:
class Classified_mean_test_features(TestingFeature):
    def __init__(self, tG, dev_tG, dev2_tG, RUL,classifier_in):#classifier_in为训练好的一个分类器
        super().__init__(tG, dev_tG, dev2_tG, RUL,classifier_in)

    def __len__(self):
        return CFG.num_in_feature_classes
    
    def __getitem__(self,idx):
        data={
            "input":torch.tensor(np.mean(self.input[np.where(self.labels==idx)],axis=0),dtype=torch.float),
            "label":torch.tensor(idx,dtype=torch.int64)
        }
        return data
    
    

In [24]:
test_classified_dataset=Classified_mean_test_features(tG_delete,
                              dev_tG_delete,
                              dev2_tG_delete,
                              RUL_delete,classifier_in)
                                                     

In [25]:
test_classified_dataset[1]

{'input': tensor([0.1046, 0.1109, 0.0849, 0.0655, 0.0822, 0.0652, 0.0491, 0.0528, 0.0590,
         0.1001, 0.0736, 0.1026, 0.0337, 0.0362, 0.0398, 0.0648, 0.0478, 0.0679]),
 'label': tensor(1)}